In [2]:
import ROOT
from ROOT import TFile, TTree, TObject, TCanvas
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from ROOT import TGraph, gRandom

import pandas as pd
from root_pandas import read_root, to_root

import time
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.24/02


In [ ]:
#work with other files, should
#just load individual files as df_ef

In [3]:
depo_dir = '/home/steve/Desktop/ee/'

In [4]:
root_f = "ee_ttbarsignalplustau_fromDilepton_TuneCP5.root"#load your own files here

In [5]:
df_ef = read_root(depo_dir + root_f, key = "ttBar_treeVariables_step8")#load your own steps, i used df_ef so dont change it unless you wanna ctrlf another 100+ terms
print("dataframe imported!")

dataframe imported!


In [6]:
df_ef

,top_pt,top_phi,top_rapidity,top_arapidity,top_eta,top_mass,top_boosted_theta,top_boosted_eta,top_theta_v2,tbar_pt,...,gen_c_Mnk,gen_ll_cHel,gen_ll_cLab,gen_ll_kNorm,gen_ll_rNorm,entry,isTopGen,isKinReco,eventWeight,trueLevelWeight
0,122.533958,1.300956,-1.196664,1.196664,-1.682348,172.500031,1.870636,-0.304436,1.870636,81.246429,...,-0.118690,-0.907489,-0.113028,-0.377697,-0.118690,434418,1,1,66.410027,75.167435
1,41.037472,-0.656494,0.529464,0.529464,1.607935,172.500000,1.561590,0.009206,1.561590,58.604706,...,0.016388,0.403097,0.437601,0.911363,0.016388,434441,1,1,7.390667,8.071794
2,52.381462,1.607415,1.141019,1.141019,2.279817,172.499985,0.331967,1.786623,0.331967,93.101112,...,-0.608442,0.119212,-0.566335,-0.687803,-0.608442,434465,1,1,77.069023,76.663116
3,24.183966,-3.076564,0.058302,0.058302,0.408690,172.500000,0.472183,1.424711,0.472183,53.476200,...,-0.138109,0.922472,0.382992,-0.295976,-0.138109,434490,1,1,76.395515,81.842064
4,191.157669,-0.545328,-0.466412,0.466412,-0.612288,172.499985,1.179872,0.401279,1.179872,43.931396,...,0.182348,-0.919740,0.175520,0.128638,0.182348,434635,1,1,71.350548,81.842064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97717,99.862846,-2.975827,-0.102668,0.102668,-0.203865,172.499985,1.428470,0.142809,1.428470,129.589661,...,0.538089,-0.011409,0.128391,-0.836727,0.538089,38807121,1,1,75.413277,85.823898
97718,197.406448,0.963623,-1.577669,1.577669,-1.843166,172.499908,2.148166,-0.612411,2.148166,268.018341,...,0.586242,0.759554,0.150771,0.067405,0.586242,38807198,1,1,82.560738,78.377274
97719,45.799992,0.565791,1.147503,1.147503,2.409577,172.500046,1.302960,0.271097,1.302960,41.253838,...,0.460642,0.352324,0.710124,0.054460,0.460642,38807360,1,1,65.803795,76.663116
97720,242.815384,-2.764153,0.621002,0.621002,0.741756,172.499969,1.217269,0.361130,1.217270,235.534668,...,-0.977647,-0.052118,-0.403320,-0.196986,-0.977647,38807395,1,1,69.970665,76.646706


In [7]:
#df_ef['_eta']= df_ef['_eta']-df_ef['_eta']#model

In [9]:
#all delta eta
df_ef['lb_delta_eta']= df_ef['l_eta']-df_ef['b_eta']
df_ef['lbbar_delta_eta']= df_ef['l_eta']-df_ef['bbar_eta']#model
df_ef['lnu_delta_eta']= df_ef['l_eta']-df_ef['nu_eta']#model
df_ef['lnubar_delta_eta']= df_ef['l_eta']-df_ef['nubar_eta']#model
df_ef['lbarb_delta_eta']= df_ef['lbar_eta']-df_ef['b_eta']
df_ef['lbarbbar_delta_eta']= df_ef['lbar_eta']-df_ef['bbar_eta']#model
df_ef['lbarnu_delta_eta']= df_ef['lbar_eta']-df_ef['nu_eta']#model
df_ef['lbarnubar_delta_eta']= df_ef['lbar_eta']-df_ef['nubar_eta']#model
df_ef['bnu_delta_eta']= df_ef['b_eta']-df_ef['nu_eta']#model
df_ef['bnubar_delta_eta']= df_ef['b_eta']-df_ef['nubar_eta']#model
df_ef['bbarnu_delta_eta']= df_ef['bbar_eta']-df_ef['nu_eta']#model
df_ef['bbarnubar_delta_eta']= df_ef['bbar_eta']-df_ef['nubar_eta']#model

In [10]:
#all delta Phi
df_ef['lb_delta_phi']= df_ef['l_phi']-df_ef['b_phi']
df_ef['lbbar_delta_phi']= df_ef['l_phi']-df_ef['bbar_phi']#model
df_ef['lnu_delta_phi']= df_ef['l_phi']-df_ef['nu_phi']#model
df_ef['lnubar_delta_phi']= df_ef['l_phi']-df_ef['nubar_phi']#model
df_ef['lbarb_delta_phi']= df_ef['lbar_phi']-df_ef['b_phi']
df_ef['lbarbbar_delta_phi']= df_ef['lbar_phi']-df_ef['bbar_phi']#model
df_ef['lbarnu_delta_phi']= df_ef['lbar_phi']-df_ef['nu_phi']#model
df_ef['lbarnubar_delta_phi']= df_ef['lbar_phi']-df_ef['nubar_phi']#model
df_ef['bnu_delta_phi']= df_ef['b_phi']-df_ef['nu_phi']#model
df_ef['bnubar_delta_phi']= df_ef['b_phi']-df_ef['nubar_phi']#model
df_ef['bbarnu_delta_phi']= df_ef['bbar_phi']-df_ef['nu_phi']#model
df_ef['bbarnubar_delta_phi']= df_ef['bbar_phi']-df_ef['nubar_phi']#model

In [1]:
#turn into abs

In [11]:
#abs eta
df_ef['lb_delta_eta']= df_ef['lb_delta_eta'].abs()
df_ef['lbbar_delta_eta']= df_ef['lbbar_delta_eta'].abs()
df_ef['lnu_delta_eta']= df_ef['lnu_delta_eta'].abs()
df_ef['lnubar_delta_eta']= df_ef['lnubar_delta_eta'].abs()
df_ef['lbarb_delta_eta']= df_ef['lbarb_delta_eta'].abs()
df_ef['lbarbbar_delta_eta']= df_ef['lbarbbar_delta_eta'].abs()
df_ef['lbarnu_delta_eta']= df_ef['lbarnu_delta_eta'].abs()
df_ef['lbarnubar_delta_eta']= df_ef['lbarnubar_delta_eta'].abs()
df_ef['bnu_delta_eta']= df_ef['bnu_delta_eta'].abs()
df_ef['bnubar_delta_eta']= df_ef['bnubar_delta_eta'].abs()
df_ef['bbarnu_delta_eta']= df_ef['bbarnu_delta_eta'].abs()
df_ef['bbarnubar_delta_eta']= df_ef['bbarnubar_delta_eta'].abs()

In [12]:
#abs phi
df_ef['lb_delta_phi']= df_ef['lb_delta_phi'].abs()
df_ef['lbbar_delta_phi']= df_ef['lbbar_delta_phi'].abs()
df_ef['lnu_delta_phi']= df_ef['lnu_delta_phi'].abs()
df_ef['lnubar_delta_phi']= df_ef['lnubar_delta_phi'].abs()
df_ef['lbarb_delta_phi']= df_ef['lbarb_delta_phi'].abs()
df_ef['lbarbbar_delta_phi']= df_ef['lbarbbar_delta_phi'].abs()
df_ef['lbarnu_delta_phi']= df_ef['lbarnu_delta_phi'].abs()
df_ef['lbarnubar_delta_phi']= df_ef['lbarnubar_delta_phi'].abs()
df_ef['bnu_delta_phi']= df_ef['bnu_delta_phi'].abs()
df_ef['bnubar_delta_phi']= df_ef['bnubar_delta_phi'].abs()
df_ef['bbarnu_delta_phi']= df_ef['bbarnu_delta_phi'].abs()
df_ef['bbarnubar_delta_phi']= df_ef['bbarnubar_delta_phi'].abs()

In [13]:
df_ef#check

,top_pt,top_phi,top_rapidity,top_arapidity,top_eta,top_mass,top_boosted_theta,top_boosted_eta,top_theta_v2,tbar_pt,...,lnu_delta_phi,lnubar_delta_phi,lbarb_delta_phi,lbarbbar_delta_phi,lbarnu_delta_phi,lbarnubar_delta_phi,bnu_delta_phi,bnubar_delta_phi,bbarnu_delta_phi,bbarnubar_delta_phi
0,122.533958,1.300956,-1.196664,1.196664,-1.682348,172.500031,1.870636,-0.304436,1.870636,81.246429,...,3.664300,5.247219,1.886725,2.452430,0.215773,1.367145,2.102498,0.519580,2.236657,3.819575
1,41.037472,-0.656494,0.529464,0.529464,1.607935,172.500000,1.561590,0.009206,1.561590,58.604706,...,2.616607,2.842055,2.508073,2.777720,1.292706,1.518155,3.800779,4.026227,1.485014,1.259565
2,52.381462,1.607415,1.141019,1.141019,2.279817,172.499985,0.331967,1.786623,0.331967,93.101112,...,1.233505,1.784331,0.154567,5.119201,2.629722,3.180547,2.475155,3.025981,2.489479,1.938653
3,24.183966,-3.076564,0.058302,0.058302,0.408690,172.500000,0.472183,1.424711,0.472183,53.476200,...,2.495975,3.301652,0.976984,3.188871,2.601468,3.407145,3.578452,4.384129,0.587404,0.218274
4,191.157669,-0.545328,-0.466412,0.466412,-0.612288,172.499985,1.179872,0.401279,1.179872,43.931396,...,3.837617,4.687695,0.464101,1.338921,0.758297,1.608375,1.222399,2.072476,2.097218,2.947296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97717,99.862846,-2.975827,-0.102668,0.102668,-0.203865,172.499985,1.428470,0.142809,1.428470,129.589661,...,3.310601,3.084955,0.235160,1.940516,4.381040,4.155393,4.616199,4.390553,2.440524,2.214877
97718,197.406448,0.963623,-1.577669,1.577669,-1.843166,172.499908,2.148166,-0.612411,2.148166,268.018341,...,2.838812,0.561753,1.288679,0.758161,0.401717,2.998848,0.886962,4.287527,1.159878,2.240686
97719,45.799992,0.565791,1.147503,1.147503,2.409577,172.500046,1.302960,0.271097,1.302960,41.253838,...,1.062148,2.142953,1.686344,1.356847,1.921451,3.002256,3.607795,4.688600,3.278298,4.359103
97720,242.815384,-2.764153,0.621002,0.621002,0.741756,172.499969,1.217269,0.361130,1.217270,235.534668,...,4.348174,1.368332,5.126985,2.493909,0.055158,2.924685,5.182143,2.202300,2.549067,0.430775
